In [ ]:
#selenium
#py2exe
#pyinstaller
#The elements of statistical learning
#Introduction to statistical learning
#

In [47]:
from tweepy import OAuthHandler 
import tweepy 
import json
import pickle
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")
import folium
from folium.plugins import MarkerCluster

In [2]:
logins = []
with open('access_codes.txt', 'r') as file:
    for line in file.read().splitlines():
        logins.append(line.rstrip().split('=')[1])
username, consumer_key, consumer_secret, access_token, access_token_secret = logins

In [3]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [4]:
cities = dict()
cities['Krakow'] = (str(geolocator.geocode('Kraków').latitude) + ',' + str(geolocator.geocode('Kraków').longitude))
cities['Wroclaw'] = (str(geolocator.geocode('Wrocław').latitude) + ','  + str(geolocator.geocode('Wrocław').longitude))
cities['Warszawa'] = (str(geolocator.geocode('Warszawa').latitude) + ','  + str(geolocator.geocode('Warszawa').longitude))

In [224]:
wro_tweets = [status for status in  tweepy.Cursor(api.search, q = 'endomondo', geocode = (cities['Wroclaw'] + ',15km')).items()]
kra_tweets = [status for status in  tweepy.Cursor(api.search, q = 'endomondo', geocode = (cities['Krakow'] + ',15km')).items()]
waw_tweets = [status for status in  tweepy.Cursor(api.search, q = 'endomondo', geocode = (cities['Warszawa'] + ',15km')).items()]

In [225]:
with open('wro_tweets.txt', 'wb') as f:
    pickle.dump(wro_tweets, f)
with open('kra_tweets.txt', 'wb') as f:
    pickle.dump(kra_tweets, f)
with open('waw_tweets.txt', 'wb') as f:
    pickle.dump(waw_tweets, f)

In [19]:
with open('wro_tweets.txt', 'rb') as f:
    wro_tweets_loaded = pickle.dump(f)
with open('kra_tweets.txt', 'rb') as f:
    kra_tweets_loaded = pickle.dump(f)
with open('waw_tweets.txt', 'rb') as f:
    waw_tweets_loaded = pickle.dump(f)

TypeError: dump() missing required argument 'file' (pos 2)

In [235]:
wro_activities = []
for tweet in wro_tweets:
    wro_activities.append((tweet.created_at , tweet.text, tweet.geo))
wro_activities

[(datetime.datetime(2018, 12, 1, 13, 34, 19),
  'I just finished running 4.62 km in 34m:37s with elcia using #Endomondo #endorphins https://t.co/jlxetqNjiD',
  {'type': 'Point', 'coordinates': [51.13318289, 17.05602095]}),
 (datetime.datetime(2018, 11, 30, 15, 26, 18),
  'I just finished cycling 17.28 km in 1h:02m:55s with #Endomondo #endorphins https://t.co/EVmsDvJq22',
  None),
 (datetime.datetime(2018, 11, 30, 13, 34, 59),
  'I just finished cycling 17.83 km in 49m:05s with #Endomondo #endorphins https://t.co/jnDCWGvvsF',
  {'type': 'Point', 'coordinates': [51.14037758, 17.06803808]}),
 (datetime.datetime(2018, 11, 30, 6, 0, 38),
  'I just finished cycling 14.48 km in 41m:19s with #Endomondo #endorphins https://t.co/RN6ieIhTEF',
  {'type': 'Point', 'coordinates': [51.10817506, 16.90879256]}),
 (datetime.datetime(2018, 11, 29, 15, 51, 3),
  'I just finished cycling 13.64 km in 41m:19s with #Endomondo #endorphins https://t.co/fWuguJDycP',
  {'type': 'Point', 'coordinates': [51.1404049

In [80]:
test = [status for status in tweepy.Cursor(api.search, q = 'endomondo').items(10)]

In [81]:
len(test)

1000

In [82]:
tweets = tweepy.Cursor(api.search, q = 'endomondo')

In [142]:
activities = []
for tweet in test:
    #print((tweet._json)['place']['country'])
    if tweet.geo != None:
         activities.append((tweet.text, tweet.geo))

In [149]:
with open('test_tweets.txt', 'wb') as f:
    pickle.dump(activities, f)

In [6]:
with open('test_tweets.txt', 'rb') as f:
    new = pickle.load(f)


In [41]:
colors = {
    'running' : 'red',
    'cycling' : 'blue'
}

In [36]:
what_I_want =  []
for tweet in new:
    if tweet.geo:
        sentence = tweet.text.split(' ')
        if 'finished' in sentence:
            what_I_want.append((sentence[sentence.index('finished') + 1], tweet.geo['coordinates']))
        else:
            what_I_want.append((sentence[sentence.index('out') + 1], tweet.geo['coordinates']))
what_I_want

[('running', [51.13318289, 17.05602095]),
 ('cycling', [51.14037758, 17.06803808]),
 ('cycling', [51.10817506, 16.90879256]),
 ('cycling', [51.14040499, 17.06800858]),
 ('running', [51.1034602, 17.0393679]),
 ('cycling', [51.10852324, 16.90817549]),
 ('cycling', [51.14034698, 17.0680095]),
 ('cycling', [51.15175559, 16.8825956]),
 ('cycling', [51.10834127, 16.90844187]),
 ('running', [51.1105186, 16.9722395]),
 ('cycling', [51.14037221, 17.06804411]),
 ('cycling', [51.10834178, 16.9084614]),
 ('running', [51.14035595, 17.06775997]),
 ('cycling', [51.14041563, 17.06807278]),
 ('cycling', [51.14040054, 17.0680929]),
 ('cycling', [51.10832585, 16.90844765]),
 ('running', [51.1039077, 17.0098738]),
 ('running', [51.12086767, 17.049736]),
 ('cycling', [51.13597029, 16.92792082]),
 ('cycling', [51.14036643, 17.06803054]),
 ('cycling', [51.109858, 17.027187]),
 ('cycling', [51.1083468, 16.90848067])]

In [49]:
m = folium.Map(
    location=[geolocator.geocode('Wrocław').latitude, geolocator.geocode('Wrocław').longitude],
    zoom_start=12,
    tiles='Stamen Terrain'
)

marker_cluster = MarkerCluster().add_to(m)

for activity in what_I_want:    
    folium.Marker(
        location=activity[1],
        popup=activity[0],
        icon=folium.Icon(color = colors[activity[0]]),
    ).add_to(marker_cluster)


m

In [136]:
(test[0]._json)

{'created_at': 'Sat Dec 01 20:09:28 +0000 2018',
 'id': 1068960293320093697,
 'id_str': '1068960293320093697',
 'text': 'I was out walking 2.53 km with #Endomondo #endorphins https://t.co/lPcWsDMtEf',
 'truncated': False,
 'entities': {'hashtags': [{'text': 'Endomondo', 'indices': [31, 41]},
   {'text': 'endorphins', 'indices': [42, 53]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/lPcWsDMtEf',
    'expanded_url': 'https://goo.gl/oMEhnD',
    'display_url': 'goo.gl/oMEhnD',
    'indices': [54, 77]}]},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="https://www.endomondo.com" rel="nofollow">Endomondo</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 356110640,
  'id_str': '356110640',
  'name': 'Ireneusz Harazim',
  'screen_name': 'IreneuszHarazim',
  'location': 'Poland',
  'descri

In [ ]:
city = ['Wro', 'Crac'] -> 'lat', 'long'  #this in string
# add +15km

In [61]:
tweepy.Cursor(api.search, q = 'endomondo')#, geocode = 'string with coordinates')